In [1]:
USE_LOCAL = False #set this to True to use model created by running train.ipynb, and to False to download models from gdrive

In [2]:
import requests
import os
from io import BytesIO
import shutil
if not os.path.exists("data"):
    os.makedirs("data")

files = ["CHMF Акции.csv",
         "MAGN Акции.csv",
         "NLMK Акции.csv",
         "test.xlsx",
         "train.xlsx",
         "Грузоперевозки.xlsx",
         "Данные рынка стройматериалов.xlsx",
         "Индекс LME.xlsx",
         "Макропоказатели.xlsx",
         "Показатели рынка металла.xlsx",
         "Топливо.xlsx",
         "Цены на сырье.xlsx"]

url = "https://raw.githubusercontent.com/samoletpanfilov/reinforcement_task/master/data/"
for f in files:
    with open("data/"+f, 'wb') as out_file:
        shutil.copyfileobj(BytesIO(requests.get(url+f).content), out_file)

In [3]:
!pip install catboost gdown

In [4]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import os
import gdown
url = "https://drive.google.com/drive/folders/1xW3Pt8H8I4AwnPb7bJ7ueouENRaU8Dzs?usp=sharing"
if not USE_LOCAL:
    gdown.download_folder(url, quiet=False, use_cookies=False)

horizon = 10
month_length = 30
inf = 1000000000
test_size = 28

Retrieving folder list


Processing file 1PN90dKZvgNe1Wph1AgNa9L0EGhDh_l7M columns_with_high_nan.txt
Retrieving folder 1GwOuQu6U10GCz7bxTjQjvxkP_uOYrLkJ test1
Processing file 16DDHR9hMAJQ29dChUdtzc5VfBF7qRSro cb_model_1.cbm
Processing file 1TZ85xx42Kjeht3gD61HAUmGFjZl-Ftza cb_model_2.cbm
Processing file 1HEvzbQk-uo8qYU3OA4NAl0cRaM5hfBoG cb_model_3.cbm
Processing file 1JndH6NBc03DqVN3RpwODcLNqEIqFrstd cb_model_4.cbm
Processing file 1PTHYYs55HFWcofzF1divGdd9byn69Hix cb_model_5.cbm
Processing file 10msd1jRX6xP6_IRjuV47fAZLX4Qk1LKj cb_model_6.cbm
Processing file 1UxuqC_lUq92UAoLzUC9zWmdVW7iIXc3F cb_model_7.cbm
Processing file 1E9KyhFRF38Cc0ryL9SpAFqhpGY4pcI2x cb_model_8.cbm
Processing file 1c1OmdX3E428u8FjYgjFMP5xFzEu8G1Sx cb_model_9.cbm
Processing file 1uLL7Jneo2Hx1kXdmdIcS1KXPA6HoFoL4 test1.mmm
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1PN90dKZvgNe1Wph1AgNa9L0EGhDh_l7M
To: c:\Users\Vitya\Desktop\Новая папка\models\columns_with_high_nan.txt
100%|██████████| 2.35k/2.35k [00:00<?, ?B/s]
Downloading...
From: https://drive.google.com/uc?id=16DDHR9hMAJQ29dChUdtzc5VfBF7qRSro
To: c:\Users\Vitya\Desktop\Новая папка\models\test1\cb_model_1.cbm
100%|██████████| 1.17M/1.17M [00:00<00:00, 5.37MB/s]
Downloading...
From: https://drive.google.com/uc?id=1TZ85xx42Kjeht3gD61HAUmGFjZl-Ftza
To: c:\Users\Vitya\Desktop\Новая папка\models\test1\cb_model_2.cbm
100%|██████████| 1.17M/1.17M [00:00<00:00, 5.50MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HEvzbQk-uo8qYU3OA4NAl0cRaM5hfBoG
To: c:\Users\Vitya\Desktop\Новая папка\models\test1\cb_model_3.cbm
100%|██████████| 1.17M/1.17M [00:00<00:00, 5.64MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JndH6NBc03DqVN3RpwODcLNqEIqFrstd
To: c:\Users\Vitya\Desktop\Нова

In [5]:
def load_merge_data() :
    df_train = pd.read_excel(
    "data/train.xlsx").rename(columns={"dt": "timestamp", "Цена на арматуру": "target"})
    df_test = pd.read_excel(
        "data/test.xlsx").rename(columns={"dt": "timestamp", "Цена на арматуру": "target"})
    df_full = pd.concat([df_train, df_test], ignore_index=True)
    df_chmf = pd.read_csv(
        "data/CHMF Акции.csv").rename(columns={"Date": "timestamp"})
    df_magn = pd.read_csv(
        "data/MAGN Акции.csv").rename(columns={"Дата": "timestamp"})
    df_nlmk = pd.read_csv(
        "data/NLMK Акции.csv").rename(columns={"Date": "timestamp"})
    df_transfer = pd.read_excel("data/Грузоперевозки.xlsx").rename(
        columns={"dt": "timestamp", "Индекс стоимости грузоперевозок": "transfer_cost"})
    df_market = pd.read_excel(
        "data/Данные рынка стройматериалов.xlsx").rename(columns={"dt": "timestamp"})
    df_lme = pd.read_excel(
        "data/Индекс LME.xlsx").rename(columns={"дата": "timestamp"})
    df_macro = pd.read_excel(
        "data/Макропоказатели.xlsx").rename(columns={"dt": "timestamp"})
    df_fuel = pd.read_excel(
        "data/Топливо.xlsx").rename(columns={"dt": "timestamp"})
    df_raw_prices = pd.read_excel(
        "data/Цены на сырье.xlsx").rename(columns={"dt": "timestamp"})

    df_chmf["timestamp"] = pd.to_datetime(df_chmf["timestamp"])
    df_magn["timestamp"] = pd.to_datetime(df_magn["timestamp"])
    df_nlmk["timestamp"] = pd.to_datetime(df_nlmk["timestamp"])

    # Merge the dataframes
    merged_df = pd.merge(df_full, df_chmf, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_magn, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_nlmk, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_transfer, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_market, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_lme, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_macro, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_fuel, on='timestamp', how='outer')
    merged_df = pd.merge(merged_df, df_raw_prices, on='timestamp', how='outer')

    merged_df = merged_df.sort_values("timestamp")
    return merged_df

In [6]:
def prepare_data(merged_df):
    for window in range(3, 2*month_length): #target over a rolling window
        merged_df['EMA'+str(window)] = merged_df['target'].ewm(alpha=2 / (window + 1), adjust=False).mean()

    merged_df = merged_df.dropna(subset=["target"])



    # Iterate over the columns in the DataFrame
    for column in merged_df.columns:
        if column != "timestamp":
            # Check if the column contains non-numeric values
            if merged_df[column].dtype != float:
                # Extract numeric values using regular expressions
                merged_df[column] = merged_df[column].apply(lambda x: re.findall(
                    r"[-+]?\d*\.\d+|\d+", str(x))[0] if re.findall(r"[-+]?\d*\.\d+|\d+", str(x)) else None)

                # Convert the column to float data type
                merged_df[column] = merged_df[column].astype(float)

    # Convert the "timestamp" column to datetime, if needed
    merged_df["timestamp"] = pd.to_datetime(merged_df["timestamp"])

    with open("models/columns_with_high_nan.txt", "r", encoding='utf8') as file:
        columns_to_drop = file.read().splitlines()

    merged_df = merged_df.drop(columns=columns_to_drop)

    return merged_df

In [7]:
def make_data_blocks(df, shift, window): # make a new dataset with data between weeks <t-shift-window> and <t-shift>, to predict the change of price between week <t> and week <t-shift>
    upgrade_df = df[["timestamp"]].copy()
    rga, rgb = shift, shift+window

    x = np.array(df["target"]-df["target"].shift(rga)) / 100 # TODO
    x = x / (np.abs(x)+1) #SoftSign
    upgrade_df["target"] = x

    for col in df.columns:
        if col !="timestamp":
            for i in range(rga, rgb):
                col1 = col+str(i)
                if i!=rga:
                    upgrade_df[col1] = df[col].shift(i)-df[col].shift(rga)
                else:
                    upgrade_df[col1] = df[col].shift(i)

    upgrade_df = upgrade_df[horizon:]
    return upgrade_df

In [8]:
def train_test_split(df): #cut test data from the end of the dataset
    return (df[:-test_size].copy().reset_index(drop=True), df[-test_size:].copy().reset_index(drop=True))

In [9]:
class model: #contains the model, and data, prepared for it (different models require different states of data)
    def __init__(self, horizon, window, name, is_full_data_train = False):
        df = prepare_data(load_merge_data())
        upgrade_df = make_data_blocks(df, horizon, window)
        if is_full_data_train:
            upgrade_train = upgrade_df
            upgrade_test = upgrade_df
        else:
            upgrade_train, upgrade_test = train_test_split(upgrade_df)
        self.test_y = upgrade_test['target']
        self.test_x = upgrade_test.drop(columns=['target', "timestamp"])
        self.train_y = upgrade_train['target']
        self.train_x = upgrade_train.drop(columns=['target', "timestamp"])
        self.name = name
        self.model = CatBoostRegressor(verbose=0)

    def save(self):
        self.model.save_model(self.name)

    def load(self):

        self.model.load_model(self.name)

    def fit(self):
        self.model = CatBoostRegressor(verbose=0)#, eval_metric = "R2")
        self.model.fit(self.train_x , self.train_y)

    def predict(self):
        return self.model.predict(self.test_x)

    def true(self):
        return self.test_y

In [10]:
def decision_prices(test):
    test = test.set_index('dt')
    tender_price = test['Цена на арматуру']
    decision = test['Объем']
    start_date = test.index.min()
    end_date = test.index.max()

    _results = []
    _active_weeks = 0
    for report_date in pd.date_range(start_date, end_date, freq='W-MON'):
        if _active_weeks == 0:  # Пришла пора нового тендера
            _fixed_price = tender_price.loc[report_date]
            _active_weeks = int(decision.loc[report_date])
        _results.append(_fixed_price)
        _active_weeks += -1
    cost = sum(_results)
    return cost # Возвращаем затраты на периоде

In [11]:
class MetaModel:
    def __init__(self, window, name):
        self.cb_models = []
        self.risk_value = 0
        self.name = name
        self.cnt_models = horizon-1
        if not os.path.exists("models/"+ name):
            os.mkdir("models/"+ name)
        for i in range(1, horizon):
            self.cb_models.append(model(i, window, "models/"+ name + "/cb_model_" + str(i)+".cbm"))


    def save(self):
        for _model in self.cb_models:
            _model.save()
        # write risk_value in file
        with open("models/"+self.name+"/"+ self.name+".mmm", "w", encoding='utf8') as file:
            file.write(str(self.risk_value)+"\n")
            file.write(str(self.cnt_models)+"\n")


    def load(self):
        for _model in self.cb_models:
            _model.load()
        with open("models/"+ self.name+"/"+self.name+".mmm", "r", encoding='utf8') as file:
            f = file.read().splitlines()
            self.risk_value, self.cnt_models  = float(f[0]), int(f[1])


    def __find_suitable_index(self, lst):
        for i in range(1, lst.__len__()):
            lst[i]= (lst[0]* self.risk_value+lst[i]*(1- self.risk_value))
        for i, num in enumerate(lst):
            if num < 0:
                return i
        return len(lst)


    def predict(self):
        result = []
        step = 0
        models_predictions = [_model.predict() for _model in self.cb_models]
        test_size = models_predictions[0].__len__()
        for day_ind in range(test_size):
            if step == 0:
                segment = [models_predictions[i][day_ind] for i in range(self.cnt_models)]
                step = self.__find_suitable_index(segment) + 1
                result.append(min(step, test_size - day_ind))
            else:
                result.append(0)
            step-=1
        return result

In [12]:
window = 10 # this parameter was calculated in train.ipynb, but it is needed for data preprocessing, so you can not save it in the model
final_model = MetaModel(window, "test1")
final_model.load()

In [13]:
res = final_model.predict()
print(*res)
test = pd.read_excel('./data/test.xlsx')
test['Объем'] = res
test.to_excel('submission.xlsx', index=False)

1 1 1 1 1 1 1 1 1 1 1 1 1 8 0 0 0 0 0 0 0 7 0 0 0 0 0 0


In [14]:
decision_prices(test) # score on test

1189100